#  Coevolução no jogo Rastros
## Projeto nº 3
### Introdução à Inteligência Artificial - edição 2020/21


## Grupo: 42

### Elementos do Grupo

Nome: Ivo Veiga

Número: 44865

Nome: João Silva

Número: 48782

Nome: Manuel Tovar

Número: 49522

## Relatório

### Funções características do jogo Rastros
As Funções características são a distância à base e distância ao objectivo.

In [1]:
from rastros import *

def dist(a, b):
    return max(abs(a[0]-b[0]), abs(a[1]-b[1]))

def dist_base(state,player):
    return dist(state.white, (4,5))

def dist_obj(state,player):
    goal = (8, 1)
    no_goal = (1, 8)
    if player == "N":
        goal = (1, 8)
        no_goal = (8, 1)
    return dist(state.white, goal)

#lista das funções características
list_caracts = [dist_base, dist_obj]

### Função de avaliação, jogadores com cromossomas distintos e execução de pares de jogos

A associação de um jogador a um cromossoma é feita antes da execução do primeiro jogo de um par usando um dicionário em que a chave são os jogadores obtidos pelos atributos `first` e `second` de uma subclasse de Game e os valores os cromossomas correspondentes. No segundo jogo do par de jogos os valores são trocados.

Assim, o cromossoma que a função de avaliação vai usar depende do jogador que lhe for passado como argumento e os jogadores, por serem construidos com a mesma função de avaliação, reduzem-se apenas à representação por um cromossoma.

A função de avaliação tem o seguinte aspeto:

In [2]:
#dicionario jogador-cromossoma
p_chromossome = {}
    
def f_eval(state,player):
    #estados terminais
    win_or_lose = state.compute_utility(player)
    if win_or_lose == 1:
        return infinity
    elif win_or_lose == -1:
        return -infinity
    else:
        #combinacao linear dos cromossomas peso * funcao caracteristica 
        res = 0
        for chrom_index, caract in enumerate(list_caracts):
            res += p_chromossome[player][chrom_index] * caract(state,player)
        return res

A associação de um jogadores a dois cromossomas [2,4] e [1,3] para um primeiro jogo de um par é então:

In [3]:
game_a = Rastros()
p_chromossome[game_a.first] = [2,4]
p_chromossome[game_a.second] = [1,3]

E para o segundo jogo a ordem dos jogadores é invertida:

In [4]:
game_a = Rastros()
p_chromossome[game_a.first] = [1,3]
p_chromossome[game_a.second] = [2,4]

Como a função de avaliação usada pelo alfabeta é igual para ambos os jogadores, existe só um jogador genérico:

In [5]:
limite_prof_exemplo = 2
jogador_exemplo = lambda game, state: alphabeta_cutoff_search_new(state,game,limite_prof_exemplo,eval_fn=f_eval)

Para executar um par de jogos pode então ser definida a seguinte função:

In [6]:
def play_game_pair(game_class,chromossome1,chromossome2,alfabeta_func,verbose=False) :
    def play_one_game(game):
        estado = game.initial
        if verbose :
            game.display(estado)
        fim = False
        while not fim :
            jogada = alfabeta_func(game,estado)
            if jogada == None:
                return 1 if game.to_move(estado) == game.second else -1
            estado = game.result(estado,jogada)
            if verbose :
                game.display(estado)
            fim = game.terminal_test(estado) 
        return game.utility(estado, game.to_move(game.initial))
    #primeiro jogo
    game = game_class()
    p_chromossome[game.first] = chromossome1
    p_chromossome[game.second] = chromossome2        
    res_with_chrom1_fst = play_one_game(game)    
    #segundo jogo
    game = game_class()
    p_chromossome[game.first] = chromossome2
    p_chromossome[game.second] = chromossome1        
    res_with_chrom2_fst = play_one_game(game)
    
    if res_with_chrom1_fst == res_with_chrom2_fst:
        return 1,1 #empate ambos têm 1 vitória
    elif res_with_chrom1_fst > res_with_chrom2_fst:
        return 2,0 #chrom1 tem 2 vitórias , chrom2 tem 0 vitórias
    else:
        return 0,2 #chrom1 tem 0 vitórias , chrom2 tem 2 vitórias    

Exemplo de execução de um par de jogos entre os cromossomas [3,4] e [2,3] com limite de profundidade 2 (limite_prof_exemplo definido anteriormente):

In [7]:
chrom1_wins, chrom2_wins = play_game_pair(Rastros, [3,4], [2,3], jogador_exemplo, verbose=False)
print("Vitórias do cromossoma 1: ", chrom1_wins)
print("Vitórias do cromossoma 2: ", chrom2_wins)

Vitórias do cromossoma 1:  1
Vitórias do cromossoma 2:  1


Para se executarem N pares de jogos entre dois cromossomas para cada limite de profundidade define-se a seguinte função:

In [8]:
def play_n_pairs_per_limit(game_class,chromossome1,chromossome2, num_pairs_per_limit, depth_limits):
    chrom1_res = 0
    chrom2_res = 0
    for depth in depth_limits:  
        alfabeta_func = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=f_eval)        
        for i in range (num_pairs_per_limit):
            chrom1_wins, chrom2_wins = play_game_pair(game_class, chromossome1, chromossome2, alfabeta_func)
            chrom1_res += chrom1_wins
            chrom2_res += chrom2_wins
    return chrom1_res, chrom2_res

Resultados da execução de 20 pares de jogos entre dois cromossomas ([1,-100] e [3,2]) para cada limite da lista [1,2,3,4] de limites:

In [9]:
depth_limits = [1,2,3,4]
num_pairs = 20
chrom1 = [1,-100]
chrom2 = [3,2]
chrom1_wins, chrom2_wins = play_n_pairs_per_limit(Rastros, chrom1, chrom2, num_pairs, depth_limits)
print("Vitórias do cromossoma ", chrom1, " : ", chrom1_wins)
print("Vitórias do cromossoma ", chrom2, " : ", chrom2_wins)

Vitórias do cromossoma  [1, -100]  :  42
Vitórias do cromossoma  [3, 2]  :  118


Resultados da execução de mais 20 pares de jogos entre os dois cromossomas anteriores, agora com o cromossoma [3,2] como o cromossoma1 e [1,-100] como cromossoma2: 

In [10]:
chrom1 = [3,2]
chrom2 = [1,-100]
chrom1_wins, chrom2_wins = play_n_pairs_per_limit(Rastros, chrom1, chrom2, num_pairs, depth_limits)
print("Vitórias do cromossoma ", chrom1, " : ", chrom1_wins)
print("Vitórias do cromossoma ", chrom2, " : ", chrom2_wins)

Vitórias do cromossoma  [3, 2]  :  113
Vitórias do cromossoma  [1, -100]  :  47


Para comprovar que a função de avaliação e a execução dos jogos estão bem implementadas e integram corretamente os cromossomas é necessário que os resultados anteriores sejam consistentes, ou seja que se observe que o número de vitórias em relação ao total de jogos para um dado cromossoma seja aproximadamente igual quer ou não esse cromossoma seja o primeiro argumento da função de execução de jogos. Isto é o caso, já que o cromossoma [3,2] é consistentemente melhor que o [1,-100].

### População inicial
Cada cromossoma da população inicial terá características representadas por inteiros pré-determinados. Ou seja, a gene pool será previamente gerada.

Define-se então a função que cria a população inicial:

In [11]:
def init_population(dim, gene_pool, num_caracts):
    assert (dim & (dim-1) == 0) and dim != 0 #dim tem de ser potência de 2 (100 (4) and 011 (3) = 000)
    g = len(gene_pool)
    population = []
    for i in range(dim):
        new_individual = [gene_pool[random.randrange(0, g)] for c in range(num_caracts)]
        population.append(new_individual)
    return population

Usando o intervalo [-1000,1000] para os valores possíveis da gene pool, a população inicial pode ser criada da seguinte maneira:

In [12]:
gene_pool = [i for i in range(-1000,1001,1)]
pop_size = 8
population = init_population(pop_size, gene_pool, len(list_caracts))
print(population)

[[931, 426], [863, -127], [906, 877], [-540, -901], [-120, -669], [656, 599], [979, -201], [746, -106]]


### Fitness: competição tipo taça
É feita a competição tipo taça entre a população para obter uma tabela de cromossomas e respetivos fitness. Esta tabela será progressivamente construída ao longo da competição de modo a que o fitness de cada cromossoma seja igual ao número de rondas em que se mantiveram na competição. O vencedor terá fitness igual ao número total de rondas + 1.

Um exemplo de uma tabela para uma população de 8 elementos pode ser [([1, 2], 1), ([-1, -2], 1), ([-5, -3], 1), ([2, -1], 1), ([-6, -7], 2), ([-6, 9], 2), ([0, -3], 3), ([-3, -2], 4)] , onde o cromossoma [1,2] tem fitness 1 e o cromossoma [-3,-2] tem fitness 4.

Define-se a função que executa uma competição tipo taça do seguinte modo:

In [13]:
def do_cup_competition(population, game_class, num_game_pairs, depth_limits):    
    fitness_list = []
    
    def do_round(players, i_round):
        winners = []
        for i in range(int(len(players)/2)):
            c1 = players[i*2]
            c2 = players[(i*2)+1]            
            wins1, wins2 = play_n_pairs_per_limit(game_class, c1, c2, num_game_pairs, depth_limits)
            if wins1 == wins2:
                if random.choice([True,False]):
                    winners.append(c1)
                    fitness_list.append((c2,i_round))
                else:
                    winners.append(c2)
                    fitness_list.append((c1,i_round))
            elif wins1 > wins2:
                winners.append(c1)
                fitness_list.append((c2,i_round))
            else:
                winners.append(c2)
                fitness_list.append((c1,i_round))
                
        if len(winners) > 1:
            do_round(winners,i_round+1)
        else:
            fitness_list.append((winners[0], i_round+1))
    
    random.shuffle(population) #modifica var. population, devolve None
    do_round(population, 1) #constroi a fitness_list
    
    return fitness_list

Exemplo de como fazer uma competição tipo taça entre `population` previamente gerada, no jogo Rastros e com 4 pares de jogos para cada um dos limites 2 e 3 por duelo:

In [14]:
pop_fitness = do_cup_competition(population, Rastros, 4, [2,3])
print(pop_fitness)

[([-120, -669], 1), ([863, -127], 1), ([-540, -901], 1), ([746, -106], 1), ([979, -201], 2), ([656, 599], 2), ([906, 877], 3), ([931, 426], 4)]


### Seleção por torneio
explicação...

### Geração dos dois descendentes por cruzamento de 1 ponto de corte
explicação...

### Pssibilidade de mutação (em todos os genes)
explicação...

### Uso do elitismo
explicação...

### Formação da geração seguinte, dada uma geração
explicação...

### Função principal
explicação...

### Execução do algoritmo de coevolução para o jogo Rastros
explicação... executando o alfabeta para uma profundidade limitada a 2 e a 1

resultado das evoluções para o Rastros para limites de profundidades maiores  (por exemplo, 4 e 5): (melhorar este texto no fim...)

### Usando o algoritmo de coevolução para outros jogos
explicação... (tic tac toe de uma das tps, 4 características básicas do tipo: o número de centros que tenho, o número de centros que ele tem, o número de cantos que tenho, o número de cantos que ele tem)